In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
import emoji
import pandas as pd 
import numpy as np
import re
import ipywidgets as widgets
from datetime import datetime
from textblob_de import TextBlobDE as TextBlob

from tabulate import tabulate

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

start_time = datetime.now()

data = pd.read_csv("B:\Dropbox\MA\KLZ-Data\swatio-ticket-export-4033-2020-09-30.csv", delimiter=';', error_bad_lines=False)
#data.head()

In [ ]:
sub_selection = data[["Kategorie", "Von_(Name)", "Text", "Post-Link", "Erstelldatum", "Zuletzt_bearbeitet", "Gelöscht", "Versteckt"]]
#print(emoji.emojize(tabulate(sub_selection)))
#sub_selection

In [ ]:
answers_regex = "Öffentlicher Post - Antwort #(\d)+"
sub_answers_regex = "Öffentlicher Post - Antwort #(\d)+\.(\d)+"

top_accordion = None
mid_vbox = None
mid_accordions = []
mid_accordion = None
low_vbox = None
low_entries = []

editted = ""
deleted_or_hidden = ""

time_format = "%Y-%m-%d %H:%M:%S"

sentiment = ""

for i in range(int(len(sub_selection) / 10)):
    date_posted = datetime.strptime(sub_selection["Erstelldatum"][i], time_format)
    date_last_editted = datetime.strptime(sub_selection["Zuletzt_bearbeitet"][i], time_format)
    if date_posted == date_last_editted:
        # Not editted
        editted = ""
    else:
        editted = "*"
    
    if sub_selection["Gelöscht"][i] == "Ja" or sub_selection["Versteckt"][i] == "Ja":
        deleted_or_hidden = "[deleted/hidden]"
    else:
        deleted_or_hidden = ""
    
    sentiment = TextBlob(str(sub_selection["Text"][i])).sentiment
    
    output = emoji.emojize(sub_selection["Von_(Name)"][i] + deleted_or_hidden + editted + " - " + str(sub_selection["Text"][i])) + " - " + str(sentiment)
    
    if sub_selection["Kategorie"][i] == "Öffentlicher Post":
        output = output +  " - " + str(sub_selection["Post-Link"][i])
        if top_accordion != None:
            mid_vbox = widgets.VBox(mid_accordions)
            mid_accordions = []
            top_accordion.children = [mid_vbox]
            top_accordion.selected_index = None
            top_accordion
        top_accordion = widgets.Accordion()
        top_accordion.set_title(0, output)
    elif re.fullmatch(answers_regex, sub_selection["Kategorie"][i]) != None:
        if mid_accordion != None:
            low_vbox = widgets.VBox(low_entries)
            low_entries = []
            mid_accordion.children = [low_vbox]
        mid_accordion = widgets.Accordion()
        mid_accordion.set_title(0, output)
        mid_accordion.selected_index = None
        mid_accordions.append(mid_accordion)
    elif re.fullmatch(sub_answers_regex, sub_selection["Kategorie"][i]) != None:
        curr_entry = widgets.Label(value=output)
        low_entries.append(curr_entry)

In [ ]:
print("Finished")
end_time = datetime.now()
print("Duration: {}".format(end_time - start_time))